<a href="https://colab.research.google.com/github/bilalkorchii/BILALKORCHI_RNN_NLP/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==2.0.1 torchvision torchaudio torchtext==0.15.2 --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

# Text Classification with Hyperparameter Tuning using Keras Tuner

This project aims to build an optimized deep learning model for binary text classification using TensorFlow and Keras.  
We utilize an LSTM-based recurrent neural network architecture to capture sequential patterns in text data.  

Key features of this project include:  
- Data preprocessing with tokenization and padding  
- Model architecture with embedding and LSTM layers  
- Hyperparameter tuning using Keras Tuner (Hyperband) to find the best embedding dimension, number of LSTM units, learning rate, and dropout rates  
- Early stopping and learning rate scheduling to improve model generalization and prevent overfitting  
- Visualization of tuning trials and model performance metrics  

The goal is to achieve the best possible classification accuracy by systematically searching the hyperparameter space and refining the model training process.


In [2]:

import torch
from torch import nn
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import random


In [3]:
!pip install portalocker


In [4]:
!pip install --upgrade portalocker==2.7.0


  Attempting uninstall: portalocker
    Found existing installation: portalocker 3.1.1
    Uninstalling portalocker-3.1.1:
      Successfully uninstalled portalocker-3.1.1


# Load IMDB dataset with top 10,000 frequent words


In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb

vocab_size = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

print(f'Training samples: {len(x_train)}')
print(f'Test samples: {len(x_test)}')
print(f'Example review (as word indexes): {x_train[0]}')
print(f'Label (0=negative,1=positive): {y_train[0]}')


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training samples: 25000
Test samples: 25000
Example review (as word indexes): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88

# max number of words per review

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 200

x_train_padded = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test_padded = pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

print(f"Padded example (length={len(x_train_padded[0])}): {x_train_padded[0]}")


Padded example (length=200): [   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   

In [7]:
!pip install -q keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


 # build Model with keras tuner

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    model = Sequential()

    # Tune embedding dimension: 16, 32, 64
    embedding_dim = hp.Choice('embedding_dim', values=[16, 32, 64])
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))

    model.add(Dropout(0.4))  # Dropout after embedding

    # Tune number of GRU units: 16, 32, 64
    rnn_units = hp.Choice('rnn_units', values=[16, 32, 64])
    model.add(GRU(rnn_units, recurrent_dropout=0.2, return_sequences=False))

    model.add(Dropout(0.4))  # Dropout after GRU

    model.add(Dense(1, activation='sigmoid'))

    # Tune learning rate between 1e-4 and 1e-2
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt


tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=5,        # small to avoid long runs
    factor=3,
    directory='my_dir',
    project_name='my_project'
)



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
import os

import tensorflow as tf
tuner.search(
    x_train_padded,
    y_train,
    epochs=10,
    validation_split=0.2,
    batch_size=8,          # or even 8 if needed
    callbacks=[stop_early]
)






Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
16                |16                |embedding_dim
64                |64                |rnn_units
0.0027495         |0.0027495         |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
 202/2500 ━━━━━━━━━━━━━━━━━━━━ 12:11 318ms/step - accuracy: 0.5084 - loss: 0.6948

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-1c184b55ef20>", line 5, in <cell line: 0>
    tuner.search(
  File "/usr/local/lib/python3.11/dist-packages/keras_tuner/src/engine/base_tuner.py", line 234, in search
    self._try_run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.11/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.11/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
   

TypeError: object of type 'NoneType' has no len()

 # geting the best MOdel params

In [ ]:
best_hp = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters:")
print(f"Embedding dimension: {best_hp.get('embedding_dim')}")
print(f"RNN units: {best_hp.get('rnn_units')}")
print(f"Learning rate: {best_hp.get('learning_rate'):.5f}")



 # Overfitting !!! Need to be solved from the results!!

In [ ]:
for trial in trials:
    trial_id = trial.trial_id
    hp = trial.hyperparameters.values

    # Get metric histories:
    train_acc = trial.metrics.get_history('accuracy')    # training accuracy
    val_acc = trial.metrics.get_history('val_accuracy')  # validation accuracy
    train_loss = trial.metrics.get_history('loss')       # training loss
    val_loss = trial.metrics.get_history('val_loss')     # validation loss

    # Extract just the metric values:
    train_acc_vals = [m.value[0] if isinstance(m.value, (list,tuple)) else m.value for m in train_acc]
    val_acc_vals = [m.value[0] if isinstance(m.value, (list,tuple)) else m.value for m in val_acc]
    train_loss_vals = [m.value[0] if isinstance(m.value, (list,tuple)) else m.value for m in train_loss]
    val_loss_vals = [m.value[0] if isinstance(m.value, (list,tuple)) else m.value for m in val_loss]

    print(f"\nTrial {trial_id} - Hyperparameters: {hp}")

    print("Epoch | Train Acc | Val Acc | Train Loss | Val Loss")
    for i in range(max(len(train_acc_vals), len(val_acc_vals), len(train_loss_vals), len(val_loss_vals))):
        ta = f"{train_acc_vals[i]:.4f}" if i < len(train_acc_vals) else "-"
        va = f"{val_acc_vals[i]:.4f}" if i < len(val_acc_vals) else "-"
        tl = f"{train_loss_vals[i]:.4f}" if i < len(train_loss_vals) else "-"
        vl = f"{val_loss_vals[i]:.4f}" if i < len(val_loss_vals) else "-"
        print(f"{i+1:5d} | {ta:9} | {va:7} | {tl:10} | {vl:8}")


 # adding the best Hp and adding Drop out for Overfitting/regularizer

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=hp.get('embedding_dim'), input_length=maxlen))
    model.add(Dropout(0.3))  # Add dropout after embedding to prevent overfitting
    model.add(LSTM(hp.get('rnn_units'), return_sequences=False))
    model.add(Dropout(0.3))  # Dropout after LSTM
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=hp.get('learning_rate'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Build best model
best_model = build_model(best_hp)

# Early stopping callback to avoid overfitting
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the best model
history = best_model.fit(
    x_train_padded, y_train,
    epochs=80,
    validation_split=0.2,
    callbacks=[stop_early],
    batch_size=16
)


# Training the updated model



# Testing: better tokenizer: splits punctuation properly

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def encode_review(text):
    tokens = text_to_word_sequence(text)
    encoded = [word_index.get(word, 2) + 3 for word in tokens]
    return pad_sequences([encoded], maxlen=maxlen, padding='post')

new_review = "Prices are way too high for the quality offered. Not worth shopping here"
encoded_review = encode_review(new_review)
prediction = best_model.predict(encoded_review)[0][0]

confidence = prediction * 100
sentiment = "Positive" if prediction >= 0.5 else "Negative"
bar_length = int(confidence // 5)

print(f"Review: \"{new_review}\"")
print(f"Sentiment: {sentiment} ({confidence:.2f}%)")
print("Confidence: [" + "#" * bar_length + "-" * (20 - bar_length) + "]")


# Plot training & validation loss values


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


  #   Conclusion
The model uses Bidirectional LSTM with dropout and L2 regularization to reduce overfitting.

Despite these regularization techniques, the validation loss curve showed some irregular behavior, possibly due to:

Strong regularization causing underfitting early in training.

Potential data imbalance or noisy validation split.

Learning rate being too high or batch size too small.

To address this, consider:

Reducing dropout rate and L2 regularization strength.

Using stratified train/validation splits to ensure balanced classes.

Adding learning rate scheduling such as ReduceLROnPlateau.

Experimenting with different batch sizes.

Early stopping was used to prevent overfitting, restoring the best weights based on validation loss.

Overall, validation loss fluctuations are common in NLP sequence models with regularization; monitoring both validation loss and accuracy is important.

Future steps include refining data splits, tuning hyperparameters further, and possibly increasing training data for better generalization.